# TODO: HashStack

In [1]:
from typing import Dict

import IPython.display
import pandas

import classes
import constants
import db

## Load and prepare events

In [2]:
# Establish the connection.
connection = db.establish_connection()

# Load all HashStack events.
hashstack_events = pandas.read_sql(
    sql = 
    f"""
    SELECT
        *
    FROM
        starkscan_events
    WHERE
        from_address='{constants.Protocol.HASHSTACK.value}'
    ORDER BY
        block_number, id ASC;
    """,
    con = connection,
)

# Close the connection.
connection.close()

/tmp/ipykernel_121643/3190443032.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hashstack_events = pandas.read_sql(


In [3]:
hashstack_events.set_index('id', inplace = True)

In [4]:
# TODO
print({x: len(hashstack_events[hashstack_events['key_name'] == x]) for x in hashstack_events['key_name'].unique()})
# hashstack_events

{'OwnershipTransferred': 3, 'ModuleFunctionChange': 21, 'new_deposit': 299, 'savings_apr_update': 180, 'borrow_apr_update': 90, 'deposit_apr_accrued': 434, 'deposit_added': 94, 'new_loan': 129, 'deposit_withdrawal': 196, 'loan_interest_deducted': 175, 'loan_withdrawal': 48, 'loan_swap': 174, 'dial_parameters': 18, 'collateral_added': 32, 'loan_repaid': 91, 'collateral_withdrawal': 90, 'RoleGranted': 4, 'RoleAdminChanged': 3, 'liquidationState': 2, 'liquidated': 2, 'smart_liquidation': 2, 'RoleRevoked': 1}


In [5]:
hashstack_events.loc[hashstack_events['block_number'] == 31026, 'data'].iloc[0]

['0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7',
 '0x1828804c87a434',
 '0x0',
 '0x3b8b11827e2434',
 '0x0',
 '0x0',
 '0x0',
 '0x1',
 '0x0',
 '0x17',
 '0x23629135f68000',
 '0x0',
 '0x642aad3a']

In [6]:
# TODO
import decimal

event_data = hashstack_events.loc[hashstack_events['key_name'] == 'new_loan', 'data'].iloc[0]
print(decimal.Decimal(str(int(event_data[5], base=16))))
event_data

0


['0x1',
 '0x5b55db55f5884856860e63f3595b2ec6b2c9555f3f507b4ca728d8e427b7864',
 '0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7',
 '0x0',
 '0x4a9b6384488000',
 '0x0',
 '0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7',
 '0x4a884a12be3000',
 '0x0',
 '0x2',
 '0x1',
 '0x6e6f6e65',
 '0x6405b735',
 '0x49d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7',
 '0x27147114878000',
 '0x0',
 '0x27147114878000',
 '0x0',
 '0x0',
 '0x0',
 '0x1',
 '0x0',
 '0x6405b735']

# Process events

TODO

Events examples:
- 'OwnershipTransferred': not needed?
- 'ModuleFunctionChange': not needed?
- 'dial_parameters': not needed?
- 'savings_apr_update': not needed?
- 'borrow_apr_update': not needed?
- 'RoleGranted': not needed?
- 'RoleAdminChanged': not needed?
- 'RoleRevoked': not needed?

- 'loan_interest_deducted': not needed? assuming that this can be ignored since final amounts are pulished in the other events

- 'liquidationState': not needed?
- 'smart_liquidation': not needed?

- [new_deposit](https://starkscan.co/event/0x07f3dbd47f1f73f88e4738952b926ca8f1ce3d0e3ab7b06274563aa87c96861d_2): not needed?
- [deposit_added](https://starkscan.co/event/0x008298bf990ac4988b889264883bfb082b5a8e3638e557a89e07a919f9faa8a7_3): not needed?
- [deposit_apr_accrued](https://starkscan.co/event/0x0471d0d9056c3623e4f76ff4f3a26d808c3f7d6b7f36d098eb3ceca9be89641b_0): not needed?
- [deposit_withdrawal](https://starkscan.co/event/0x02a2541942d3fe4f7af4b578f87aa13507beff31db954847dd3b8e9600904903_3): not needed?

- [new_loan](https://starkscan.co/event/0x04ff9acb9154603f1fc14df328a3ea53a6c58087aaac0bfbe9cc7f2565777db8_2)
- [loan_withdrawal](https://starkscan.co/event/0x05bb8614095fac1ac9b405c27e7ce870804e85aa5924ef2494fec46792b6b8dc_2)
- [loan_repaid](https://starkscan.co/event/0x07731e48d33f6b916f4e4e81e9cee1d282e20e970717e11ad440f73cc1a73484_1)
- [loan_swap](https://starkscan.co/event/0x00ad0b6b00ce68a1d7f5b79cd550d7f4a15b1708b632b88985a4f6faeb42d5b1_7)

- [collateral_added](https://starkscan.co/event/0x02df71b02fce15f2770533328d1e645b957ac347d96bd730466a2e087f24ee07_2) .. seems important, but doesn't include user, we need to map it using `loan_id`?
- [collateral_withdrawal](https://starkscan.co/event/0x03809ebcaad1647f2c6d5294706e0dc619317c240b5554848c454683a18b75ba_5)

- [liquidated](https://starkscan.co/event/0x0774bebd15505d3f950c362d813dc81c6320ae92cb396b6469fd1ac5d8ff62dc_8)

In [7]:
# TODO
import collections
import decimal


class HashStackBorrowings:
    """
    TODO
    """

    def __init__(self) -> None:
        self.id: int = 0
        self.owner: str = ''
        self.market: str = ''
#         self.commitment  # TODO: needed?
        self.amount: decimal.Decimal = decimal.Decimal('0')
        self.current_market: str = ''
        self.current_amount: decimal.Decimal = decimal.Decimal('0')
#         self.is_loan_withdrawn  # TODO: bool? needed?
        self.debt_category: str = ''
#         self.state  # TODO: needed?
#         self.l3_integration  # TODO: needed?
#         self.created_at  # TODO: needed?


class HashStackCollateral:
    """
    TODO
    """

    def __init__(self) -> None:
#         self.owner: str = ''  # TODO: why is this not included?
        self.market: str = ''
        self.amount: decimal.Decimal = decimal.Decimal('0')
        self.current_amount: decimal.Decimal = decimal.Decimal('0')
#         self.commitment  # TODO: needed?
#         self.timelock_validity  # TODO: needed?
#         self.is_timelock_activated  # TODO: needed?
#         self.activation_time  # TODO: needed?


class HashStackLoan:
    """
    TODO
    """

    def __init__(self) -> None:
        # TODO: save user?
        self.loan: HashStackBorrowings = HashStackBorrowings()
        self.collateral: HashStackCollateral = HashStackCollateral()


# class UserTokenState:
#     """
#     TODO
#     """

#     # TODO: make it token-dependent (advanced solution: fetch token prices in $ -> round each token's
#     #   balance e.g. to the nearest cent)
# #     MAX_ROUNDING_ERRORS = {
# #         "ETH": decimal.Decimal('0.5') * decimal.Decimal('1e13'),
# #         "wBTC": decimal.Decimal('1e2'),
# #         "USDC": decimal.Decimal('1e4'),
# #         "DAI": decimal.Decimal('1e16'),
# #         "USDT": decimal.Decimal('1e4'),
# #     }

#     def __init__(self, token: str) -> None:
#         self.token: str = token
#         # TODO: ignore deposits for now
# #         self.deposit: decimal.Decimal = decimal.Decimal("0")
#         self.collateral: decimal.Decimal = decimal.Decimal("0")
# #         self.collateral_enabled: bool = False
#         self.borrowings: decimal.Decimal = decimal.Decimal("0")
# #         self.z_token: bool = token[0] == "z"

#     def update_borrowings(self, amount: decimal.Decimal):
#         self.borrowings += amount
# #         if (
# #             -self.MAX_ROUNDING_ERRORS[self.token]
# #             < self.borrowings
# #             < self.MAX_ROUNDING_ERRORS[self.token]
# #         ):
# #             self.borrowings = decimal.Decimal("0")

#     def set_borrowings(self, amount: decimal.Decimal):
#         self.borrowings = amount
# #         if (
# #             -self.MAX_ROUNDING_ERRORS[self.token]
# #             < self.borrowings
# #             < self.MAX_ROUNDING_ERRORS[self.token]
# #         ):
# #             self.borrowings = decimal.Decimal("0")

#     def update_collateral(self, amount: decimal.Decimal):
#         self.collateral += amount
# #         if (
# #             -self.MAX_ROUNDING_ERRORS[self.token]
# #             < self.collateral
# #             < self.MAX_ROUNDING_ERRORS[self.token]
# #         ):
# #             self.collateral = decimal.Decimal("0")

#     def set_collateral(self, amount: decimal.Decimal):
#         self.collateral = amount
# #         if (
# #             -self.MAX_ROUNDING_ERRORS[self.token]
# #             < self.collateral
# #             < self.MAX_ROUNDING_ERRORS[self.token]
# #         ):
# #             self.collateral = decimal.Decimal("0")

#     # TODO: ignore deposits for now
# #     def update_deposit(self, amount: decimal.Decimal):
# #         self.deposit += amount
# # #         if (
# # #             -self.MAX_ROUNDING_ERRORS[self.token]
# # #             < self.deposit
# # #             < self.MAX_ROUNDING_ERRORS[self.token]
# # #         ):
# # #             self.deposit = decimal.Decimal("0")


class UserState:
    """
    TODO
    """

    def __init__(self) -> None:
        # TODO
        self.loans: collections.defaultdict = collections.defaultdict(HashStackLoan)
#         self.token_states: Dict[str, UserTokenState] = {
#             "ETH": UserTokenState("ETH"),
#             "wBTC": UserTokenState("wBTC"),
#             "USDC": UserTokenState("USDC"),
#             "DAI": UserTokenState("DAI"),
#             "USDT": UserTokenState("USDT"),
#         }


class State:
    """
    TODO
    """

    # TODO: f'process_{name.lower()}_event'?
    EVENTS_FUNCTIONS_MAPPING: Dict[str, str] = {
        # TODO: ignore deposits for now
#         "new_deposit": "process_new_deposit_event",
#         "deposit_added": "process_deposit_added_event",
#         "deposit_apr_accrued": "process_deposit_apr_accrued_event",
#         "deposit_withdrawal": "process_deposit_withdrawal_event",
        "new_loan": "process_new_loan_event",
        # TODO: this event shows what the user does with the loan, but it shouldn't change the amount borrowed, so let's ignore it for now
#         "loan_withdrawal": "process_loan_withdrawal_event",
        "loan_repaid": "process_loan_repaid_event",
        "loan_swap": "process_loan_swap_event",
        "collateral_added": "process_collateral_added_event",
        "collateral_withdrawal": "process_collateral_withdrawal_event",
        "liquidated": "process_liquidated_event",
    }
#     USER = '0x3139000ef6bd54e3fb2e70149e2281b660c642fdf39158fe096c9e0f662dd32'  # TODO
    USER = '0x724102a5654d669ba7ce25e420e733ddb409b753598ab0883ad476cf9faed7b'

    def __init__(self) -> None:
        self.user_loan_ids_mapping: collections.defaultdict = collections.defaultdict(list)
        self.user_states: collections.defaultdict = collections.defaultdict(UserState)
        # TODO: how to compute the interest accrued on both collateral and loan?
#         self.accumulator_states: Dict[str, AccumulatorState] = {
#             "ETH": AccumulatorState(),
#             "wBTC": AccumulatorState(),
#             "USDC": AccumulatorState(),
#             "DAI": AccumulatorState(),
#             "USDT": AccumulatorState(),
#         }

    def process_event(self, event: pandas.Series) -> None:
        # TODO: filter events in the query
        if not event["key_name"] in self.EVENTS_FUNCTIONS_MAPPING:
            return
        getattr(self, self.EVENTS_FUNCTIONS_MAPPING[event["key_name"]])(event=event)

#     def process_new_deposit_event(self, event: pandas.Series) -> None:
#         # TODO: this event signifies that it's the first deposit made by the user (is it per token or not?) -> should we assert that the deposit at the moment is 0?
#         # The order of the arguments is: `id`, `owner`, `user_record_index`, `market`, `commitment`, `amount`,
#         # `created_at`, `is_timelock_applicable`, `is_timelock_activated`, `timelock_validity`, `timestamp`.
#         user = event["data"][1]
#         token = constants.get_symbol(event["data"][3])
#         # TODO: divide by something or store like this?
#         # TODO: any better conversion to decimals?
#         # TODO: sanity checks/asserts?
#         amount = decimal.Decimal(str(int(event["data"][5], base=16)))
#         self.user_states[user].deposit(token=token, amount=amount)
#         # TODO
#         if user == self.USER:
#             print('new_deposit', token, amount)
#             print('new_deposit', event['block_number'], token)  # TODO

#     def process_deposit_added_event(self, event: pandas.Series) -> None:
#         # The order of the arguments is: `id`, `owner`, `user_record_index`, `market`, `commitment`, `amount`,
#         # `created_at`, `is_timelock_applicable`, `is_timelock_activated`, `timelock_validity`, 
#         # `activation_time`, `amount_added`, `timestamp`.
#         user = event["data"][1]
#         token = constants.get_symbol(event["data"][3])
#         # TODO: use amount or amount_added?
#         amount = decimal.Decimal(str(int(event["data"][5], base=16)))
#         self.user_states[user].deposit(token=token, amount=amount)
#         # TODO
#         if user == self.USER:
#             print('deposit_added', token, amount)
#             print('deposit_added amount_added', event['block_number'], token, decimal.Decimal(str(int(event["data"][-3], base=16))))  # TODO

#     def process_deposit_apr_accrued_event(self, event: pandas.Series) -> None:
#         # The order of the arguments is: `id`, `owner`, `user_record_index`, `market`, `commitment`, `amount`,
#         # `created_at`, `is_timelock_applicable`, `is_timelock_activated`, `timelock_validity`, 
#         # `activation_time`, `accrued_reward`, `timestamp`.
#         user = event["data"][1]
#         token = constants.get_symbol(event["data"][3])
#         # TODO: use amount or accrued_reward?
#         amount = decimal.Decimal(str(int(event["data"][5], base=16)))
#         self.user_states[user].deposit(token=token, amount=amount)
#         # TODO
#         if user == self.USER:
#             print('deposit_apr_accrued', token, amount)
#             print('deposit_apr_accrued accrued_reward', event['block_number'], token, decimal.Decimal(str(int(event["data"][-3], base=16))))  # TODO

#     def process_deposit_withdrawal_event(self, event: pandas.Series) -> None:
#         # The order of the arguments is: `id`, `owner`, `user_record_index`, `market`, `commitment`, `amount`,
#         # `created_at`, `is_timelock_applicable`, `is_timelock_activated`, `timelock_validity`, 
#         # `activation_time`, `amount_withdrawn`, `fee_deducted`, `timestamp`.
#         user = event["data"][1]
#         token = constants.get_symbol(event["data"][3])
#         # TODO: use amount or amount_withdrawn and fee_deducted?
#         amount = decimal.Decimal(str(int(event["data"][5], base=16)))
#         self.user_states[user].withdrawal(token=token, amount=-amount)
#         # TODO
#         if user == self.USER:
#             print('deposit_withdrawal', token, amount)
#             print('deposit_withdrawal amount_withdrawn', event['block_number'], token, decimal.Decimal(str(int(event["data"][-5], base=16))))  # TODO
#             print('deposit_withdrawal fee_deducted', event['block_number'], token, decimal.Decimal(str(int(event["data"][-3], base=16))))  # TODO

    def process_new_loan_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `id`, `owner`, `market`, `commitment`, `amount`, `current_market`, 
        # `current_amount`, `is_loan_withdrawn`, `debt_category`, `state`, `l3_integration`, `created_at`,
        # `market`, `amount`, `current_amount`, `commitment`, `timelock_validity`, `is_timelock_activated`,
        # `activation_time`, `timestamp`.
        loan_id = int(event["data"][0], base = 16)
        user = event["data"][1]
        self.user_loan_ids_mapping[user].append(loan_id)
        # TODO
        self.user_states[user].
#         # TODO: universal naming: use deposit, collateral and debt (instead of loan/borrowings)?
#         borrowings_token = constants.get_symbol(event["data"][2])
#         borrowings_amount = decimal.Decimal(str(int(event["data"][4], base=16)))
#         # TODO: first ~3 loans seem to have different structure of 'data'
#         try:
#             collateral_token = constants.get_symbol(event["data"][14])
#             collateral_amount = decimal.Decimal(str(int(event["data"][15], base=16)))
#         except KeyError:
#             collateral_token = constants.get_symbol(event["data"][13])
#             collateral_amount = decimal.Decimal(str(int(event["data"][14], base=16)))
#         self.user_states[user].token_states[borrowings_token].update_borrowings(borrowings_amount)
#         self.user_states[user].token_states[collateral_token].update_collateral(collateral_amount)
#         # TODO
#         if user == self.USER:
#             print(event['block_number'], 'new_loan borrowings', borrowings_token, borrowings_amount, 'collateral', collateral_token, collateral_amount)

#     def process_loan_withdrawal_event(self, event: pandas.Series) -> None:
#         # The order of the arguments is: `id`, `owner`, `market`, `commitment`, `amount`, `current_market`, 
#         # `current_amount`, `is_loan_withdrawn`, `debt_category`, `state`, `l3_integration`, `created_at`,
#         # `amount_withdrawn`, `timestamp`.
#         user = event["data"][1]
#         token = constants.get_symbol(event["data"][2])
#         # TODO: use amount or amount_withdrawn?
#         amount = decimal.Decimal(str(int(event["data"][4], base=16)))
#         # TODO: how should we interpret this event?
#         self.user_states[user].borrowing(token=token, amount=amount)
#         # TODO
#         if user == self.USER:
#             print('loan_withdrawal', token, amount)
#             print('loan_withdrawal current_amount', event['block_number'], token, decimal.Decimal(str(int(event["data"][7], base=16))))  # TODO
#             print('loan_withdrawal amount_withdrawn', event['block_number'], token, decimal.Decimal(str(int(event["data"][-3], base=16))))  # TODO

    def process_loan_repaid_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `id`, `owner`, `market`, `commitment`, `amount`, `current_market`, 
        # `current_amount`, `is_loan_withdrawn`, `debt_category`, `state`, `l3_integration`, `created_at`,
        # `timestamp`.
        user = event["data"][1]
        token = constants.get_symbol(event["data"][6])
        amount = decimal.Decimal(str(int(event["data"][7], base=16)))
        self.user_states[user].token_states[token].set_borrowings(amount)
        # TODO
        if user == self.USER:
            print(event['block_number'], 'loan_repaid', token, amount)

    def process_loan_swap_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `id`, `owner`, `market`, `commitment`, `amount`, `current_market`, 
        # `current_amount`, `is_loan_withdrawn`, `debt_category`, `state`, `l3_integration`, `created_at`,
        # `id`, `owner`, `market`, `commitment`, `amount`, `current_market`, `current_amount`,
        # `is_loan_withdrawn`, `debt_category`, `state`, `l3_integration`, `created_at`, `timestamp`.
        old_loan_id = int(event["data"][0], base = 16)
        old_user = event["data"][1]
        assert old_loan_id in self.user_loan_ids_mapping[old_user]
        new_loan_id = int(event["data"][14], base = 16)
        new_user = event["data"][15]
        assert old_loan_id == new_loan_id
        assert old_user == new_user
        # TODO: universal naming: use deposit, collateral and debt (instead of loan/borrowings)?
        old_token = constants.get_symbol(event["data"][2])
        old_amount = decimal.Decimal(str(int(event["data"][4], base=16)))  # TODO: not needed?
        new_token = constants.get_symbol(event["data"][20])
        new_amount = decimal.Decimal(str(int(event["data"][21], base=16)))
        # TODO: from the docs, we assume that the whole balance gets swapped .. is it saved in the current market/amount?
        self.user_states[new_user].token_states[old_token].set_borrowings(decimal.Decimal("0"))
        self.user_states[old_user].token_states[new_token].update_borrowings(new_amount)
#         # TODO
        if old_user == self.USER:
            print(event['block_number'], 'loan_swap old loan', old_token, old_amount, 'new loan', new_token, new_amount)

    def process_collateral_added_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `market`, `amount`, `current_amount`, `commitment`,
        # `timelock_validity`, `is_timelock_activated`, `activation_time`, `loan_id`, `amount_added`,
        # `timestamp`.
        loan_id = int(event["data"][9], base = 16)
        # TODO: create method self.find_user?
        users = [user for user, loan_ids in self.user_loan_ids_mapping.items() if loan_id in loan_ids]
        assert len(users) == 1
        user = users[0]
        # TODO: universal naming: use deposit, collateral and debt (instead of loan/borrowings)?
        token = constants.get_symbol(event["data"][0])
        final_amount = decimal.Decimal(str(int(event["data"][3], base=16)))
        self.user_states[user].token_states[token].set_collateral(final_amount)
        # TODO
        if user == self.USER:
            print(event['block_number'], 'collateral_added', token, final_amount)

    def process_collateral_withdrawal_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `market`, `amount`, `current_amount`, `commitment`,
        # `timelock_validity`, `is_timelock_activated`, `activation_time`, `loan_id`, `amount_withdrawn`,
        # `timestamp`.
        loan_id = int(event["data"][9], base = 16)
        # TODO: create method self.find_user?
        users = [user for user, loan_ids in self.user_loan_ids_mapping.items() if loan_id in loan_ids]
        assert len(users) == 1
        user = users[0]
        # TODO: universal naming: use deposit, collateral and debt (instead of loan/borrowings)?
        token = constants.get_symbol(event["data"][0])
        final_amount = decimal.Decimal(str(int(event["data"][3], base=16)))
        self.user_states[user].token_states[token].set_collateral(final_amount)
        # TODO
        if user == self.USER:
            print(event['block_number'], 'collateral_withdrawal', token, final_amount)

    def process_liquidated_event(self, event: pandas.Series) -> None:
        # The order of the arguments is: `id`, `owner`, `market`, `commitment`, `amount`, `current_market`, 
        # `current_amount`, `is_loan_withdrawn`, `debt_category`, `state`, `l3_integration`, `created_at`,
        # `liquidator`, `timestamp`.
        user = event["data"][1]
        token = constants.get_symbol(event["data"][2])
        amount = decimal.Decimal(str(int(event["data"][4], base=16)))
        # TODO: from the docs, we assume that the whole balance gets liquidated
        # TODO: what happens to the collateral? now assuming it disappears
        self.user_states[user].token_states[token].set_borrowings(decimal.Decimal("0"))
        self.user_states[user].token_states[token].set_collateral(decimal.Decimal("0"))
        # TODO
        if user == self.USER:
            print(event['block_number'], 'liquidated', token, amount)
            print('liquidated current_amount', event['block_number'], constants.get_symbol(event["data"][6]), decimal.Decimal(str(int(event["data"][7], base=16))))  # TODO

In [8]:
# Iterate over ordered events to obtain the final state of each user.
state = State()
# bns = set()  # TODO
for _, event in hashstack_events.iterrows():
#     bns.update({event['block_number']})  # TODO
    # TODO: save the timestamp/block number of each update?
    state.process_event(event = event)
# sorted([x for x in range(min(bns), max(bns)) if x + 1 not in bns])[-200:]  # TODO

38136 new_loan borrowings ETH 18000000000000000 collateral ETH 20128000000000000
38148 loan_repaid ETH 0
38148 collateral_withdrawal ETH 0
38150 new_loan borrowings USDC 82096723 collateral ETH 13678000000000000
38151 loan_swap old loan USDC 82096723 new loan ETH 40870777232421656
38451 loan_swap old loan USDC 82096723 new loan USDC 86242895
38453 collateral_withdrawal ETH 0
38453 loan_repaid USDC 0
41916 new_loan borrowings USDC 90000000 collateral USDC 30000000
41917 loan_swap old loan USDC 90000000 new loan ETH 45125154272868802
42070 collateral_withdrawal USDC 0
42070 loan_swap old loan USDC 90000000 new loan USDC 86738803
42070 loan_repaid USDC 0
43889 new_loan borrowings USDC 166528973 collateral ETH 30000000000000000
43890 loan_swap old loan USDC 166528973 new loan wBTC 464992
44793 loan_swap old loan USDC 166528973 new loan USDC 108393116
44795 liquidated USDC 166528973
liquidated current_amount 44795 USDC 0


In [9]:
# TODO
for user, user_state in state.user_states.items():
    print(user)
    for token, token_state in user_state.token_states.items():
        print(token, 'c:', token_state.collateral, 'b:', token_state.borrowings)

0x5b55db55f5884856860e63f3595b2ec6b2c9555f3f507b4ca728d8e427b7864
ETH c: 0 b: 0
wBTC c: 0 b: 0
USDC c: 15000000 b: 30000000
DAI c: 0 b: 0
USDT c: 0 b: 29614513
0x1e307316999c4830ada67b495d03246ab205e77853bb117da7b555122a33bc7
ETH c: 0 b: 12940587809644403
wBTC c: 0 b: 108059
USDC c: 0 b: 0
DAI c: 0 b: 0
USDT c: 10000000 b: 30000000
0x74061d07a0fbd8ffc376dc4f593c69854c51b6c2fe5596d88452e9efdd76864
ETH c: 0 b: 32851673213696946
wBTC c: 0 b: 0
USDC c: 0 b: 0
DAI c: 0 b: 0
USDT c: 0 b: 0
0x1caf85dbb2e8c6d551f7d94d5acc9f854680ec060f7a808f0901915ad19a127
ETH c: 0 b: 77654191645648512
wBTC c: 0 b: 160675
USDC c: 0 b: 31294416
DAI c: 0 b: 0
USDT c: 0 b: 0
0x5095078578a59f8a9c17df97188db1b59574c6d4836dd3e705fe8537624228a
ETH c: 0 b: 0
wBTC c: 0 b: 123093
USDC c: 0 b: 0
DAI c: 0 b: 57415294533117776792
USDT c: 0 b: 51155272
0x3163a2390c34b10c3ac51f063af99204f3466357e8840bc93d9997ffdb159c
ETH c: 0 b: 0
wBTC c: 0 b: 0
USDC c: 0 b: 31000000
DAI c: 0 b: 0
USDT c: 11000000 b: 0
0x732f5f56f0a0a1888a9d

In [13]:
# TODO: negative deposits?
print("Action   Token   Raw value   Raw $   Readable $")
for attr in ['collateral', 'borrowings']:
    for token in ['ETH', 'wBTC', 'USDC', 'DAI', 'USDT']:
        values = [getattr(user_state.token_states[token], attr) for user, user_state in state.user_states.items()]
        print(attr, token, pandas.Series(values).min(), f"${prices.to_dollars(pandas.Series(values).min(), token)}", prices.to_dollars_pretty(pandas.Series(values).min(), token))

Action   Token   Raw value   Raw $   Readable $
collateral ETH 0 $0E-41 $0
collateral wBTC 0 $0 $0
collateral USDC 0 $0E-53 $0
collateral DAI 0 $0E-51 $0
collateral USDT 0 $0 $0
borrowings ETH 0 $0E-41 $0
borrowings wBTC 0 $0 $0
borrowings USDC 0 $0E-53 $0
borrowings DAI 0 $0E-51 $0
borrowings USDT 0 $0 $0


In [12]:
# TODO: print out users who didn't deposit but took a loan
r = {}
for user in state.user_states:
    r[user] = {}
    for event_type in hashstack_events['key_name'].unique():
        r[user][event_type] = hashstack_events.loc[hashstack_events['key_name'] == event_type, 'data'].apply(lambda x: user in x).sum()
#         print(user, event_type, hashstack_events.loc[hashstack_events['key_name'] == event_type, 'data'].apply(lambda x: user in x).sum())
{x for x, y in r.items() if y['new_deposit'] == 0 and y['new_loan'] > 0}

{'0x2e4dff5c8d806ec7e06198066503416f872705417f53783bcaa4a3833c62881',
 '0x30ba095e7d1ef384ef89403808e96dc60ad38c5c1fd15830a63c7a130ffc408',
 '0x3163a2390c34b10c3ac51f063af99204f3466357e8840bc93d9997ffdb159c',
 '0x581e7d95b6dcb87b66205c3787461e5d7334f65f07cda9db148cef8d33c6ef5'}

In [11]:
# Get current prices of tokens.
prices = classes.Prices()

# Compute health factors

TODO: This part is new. Will be refactored.

In [ ]:
# TODO
from typing import Dict
import collections
import decimal


# Source: Starkscan, e.g. 
# https://starkscan.co/token/0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7 for ETH.
TOKEN_DECIMAL_FACTORS = {
    "ETH": decimal.Decimal('1e18'),
    "wBTC": decimal.Decimal('1e8'),
    "USDC": decimal.Decimal('1e6'),
    "DAI": decimal.Decimal('1e18'),
    "USDT": decimal.Decimal('1e6'),
}


# Source: https://zklend.gitbook.io/documentation/using-zklend/technical/asset-parameters.
COLLATERAL_FACTORS = {
    "ETH": decimal.Decimal('0.80'),
    "wBTC": decimal.Decimal('0.70'),
    "USDC": decimal.Decimal('0.80'),
    "DAI": decimal.Decimal('0.70'),
    "USDT": decimal.Decimal('0.70'),
}


# Source: https://zklend.gitbook.io/documentation/using-zklend/technical/asset-parameters.
LIQUIDATION_BONUSES = {
    "ETH": decimal.Decimal('0.10'),
    "wBTC": decimal.Decimal('0.15'),
    "USDC": decimal.Decimal('0.10'),
    "DAI": decimal.Decimal('0.10'),
    "USDT": decimal.Decimal('0.10'),
}


def compute_risk_adjusted_collateral_usd(
    user_state: classes.UserState,
    prices: Dict[str, decimal.Decimal],
) -> decimal.Decimal:
    return sum(
        token_state.collateral_enabled
        * token_state.deposit
        * COLLATERAL_FACTORS[token]
        * prices[token]
        # TODO: perform the conversion using TOKEN_DECIMAL_FACTORS sooner (in `UserTokenState`?)?
        / TOKEN_DECIMAL_FACTORS[token]
        for token, token_state
        in user_state.token_states.items()
        if not token_state.z_token
    )


def compute_borrowings_usd(
    user_state: classes.UserState,
    prices: Dict[str, decimal.Decimal],
) -> decimal.Decimal:
    return sum(
        token_state.borrowings
        * prices[token]
        # TODO: perform the conversion using TOKEN_DECIMAL_FACTORS sooner (in `UserTokenState`?)?
        / TOKEN_DECIMAL_FACTORS[token]
        for token, token_state
        in user_state.token_states.items()
        if not token_state.z_token
    )


def compute_health_factor(
    risk_adjusted_collateral_usd: decimal.Decimal,
    borrowings_usd: decimal.Decimal,
) -> decimal.Decimal:
    if borrowings_usd == decimal.Decimal('0'):
        # TODO: assumes collateral is positive
        return decimal.Decimal('Inf')

    health_factor = risk_adjusted_collateral_usd / borrowings_usd
    # TODO: enable?
#     if health_factor < decimal.Decimal('0.9'):
#         print(f'Suspiciously low health factor = {health_factor} of user = {user}, investigate.')
    # TODO: too many loans eligible for liquidation?
#     elif health_factor < decimal.Decimal('1'):
#         print(f'Health factor = {health_factor} of user = {user} eligible for liquidation.')
    return health_factor

In [ ]:
# TODO
for user, user_state in state.user_states.items():
    risk_adjusted_collateral_usd = compute_risk_adjusted_collateral_usd(
        user_state = user_state,
        prices = prices.prices,
    )
    borrowings_usd = compute_borrowings_usd(
        user_state = user_state,
        prices = prices.prices,
    )
    health_factor = compute_health_factor(
        risk_adjusted_collateral_usd = risk_adjusted_collateral_usd,
        borrowings_usd = borrowings_usd,
    )

# Compute max liquidated borrowings

TODO: This part is new. Will be refactored.

In [ ]:
# TODO
# TODO: compute_health_factor, etc. should be methods of class UserState
def compute_borrowings_to_be_liquidated(
    risk_adjusted_collateral_usd: decimal.Decimal,
    borrowings_usd: decimal.Decimal,
    borrowings_token_price: decimal.Decimal,
    collateral_token_collateral_factor: decimal.Decimal,
    collateral_token_liquidation_bonus: decimal.Decimal,
) -> decimal.Decimal:
    # TODO: commit the derivation of the formula in a document?
    numerator = borrowings_usd - risk_adjusted_collateral_usd
    denominator = borrowings_token_price * (
        1 - collateral_token_collateral_factor * (
            1 + collateral_token_liquidation_bonus
        )
    )
    return numerator / denominator


def compute_max_liquidated_amount(
    state: classes.State,
    prices: Dict[str, decimal.Decimal],
    borrowings_token: str,
) -> decimal.Decimal:
    liquidated_borrowings_amount = decimal.Decimal('0')
    for user, user_state in state.user_states.items():
        # TODO: do this?
        # Filter out users who borrowed the token of interest.
        borrowings_tokens = {
            token_state.token
            for token_state in user_state.token_states.values()
            if token_state.borrowings > decimal.Decimal('0')
        }
        if not borrowings_token in borrowings_tokens:
            continue

        # Filter out users with health below 1.
        risk_adjusted_collateral_usd = compute_risk_adjusted_collateral_usd(
            user_state = user_state,
            prices = prices,
        )
        borrowings_usd = compute_borrowings_usd(
            user_state = user_state,
            prices = prices,
        )
        health_factor = compute_health_factor(
            risk_adjusted_collateral_usd = risk_adjusted_collateral_usd,
            borrowings_usd = borrowings_usd,
        )
        # TODO
#         if health_factor >= decimal.Decimal('1') or health_factor <= decimal.Decimal('0'):
        if health_factor >= decimal.Decimal('1'):
            continue

        # TODO: find out how much of the borrowings_token will be liquidated
        collateral_tokens = {
            token_state.token
            for token_state in user_state.token_states.values()
            if token_state.deposit * token_state.collateral_enabled != decimal.Decimal('0')
        }
        # TODO: choose the most optimal collateral_token to be liquidated .. or is the liquidator indifferent?
        print(user, collateral_tokens, health_factor, borrowings_usd, risk_adjusted_collateral_usd)
        collateral_token = list(collateral_tokens)[0]
        liquidated_borrowings_amount += compute_borrowings_to_be_liquidated(
            risk_adjusted_collateral_usd = risk_adjusted_collateral_usd,
            borrowings_usd = borrowings_usd,
            borrowings_token_price = prices[borrowings_token],
            collateral_token_collateral_factor = COLLATERAL_FACTORS[collateral_token],
            collateral_token_liquidation_bonus = LIQUIDATION_BONUSES[collateral_token],
        )
    return liquidated_borrowings_amount

## Single price change

In [ ]:
# TODO
# TODO: adjust prices and observe the amounts
import copy


COLLATERAL_TOKEN = 'wBTC'
BORROWINGS_TOKEN = 'USDC'
COLLATERAL_TOKEN_PRICE = decimal.Decimal('1500')


def simulate_liquidations_under_absolute_price_change(
    prices: classes.Prices,
    collateral_token: str,
    collateral_token_price: decimal.Decimal,
    state: classes.State,
    borrowings_token: str,
) -> decimal.Decimal:
    changed_prices = copy.deepcopy(prices.prices)
    changed_prices[collateral_token] = collateral_token_price
    return compute_max_liquidated_amount(state = state, prices = changed_prices, borrowings_token = borrowings_token)


simulate_liquidations_under_absolute_price_change(
    prices = prices,
    collateral_token = COLLATERAL_TOKEN,
    collateral_token_price = COLLATERAL_TOKEN_PRICE,
    state = state,
    borrowings_token = BORROWINGS_TOKEN,
)

In [ ]:
# # TODO: debug negative deposits
# for x, y in state.user_states['0x204c44e83f63803bcae77406aa749636d23d3c914e4aa9c84f89f45bad0f844'].token_states.items():
#     print(x)
#     print(y.deposit)
#     print(y.collateral_enabled)
#     print(y.borrowings)

## Range of price changes

### Nominal price change

In [ ]:
# TODO: adjust prices and observe the amounts
import numpy


COLLATERAL_TOKEN = 'ETH'
BORROWINGS_TOKEN = 'USDC'


def decimal_range(start: decimal.Decimal, stop: decimal.Decimal, step: decimal.Decimal):
    while start < stop:
        yield start
        start += step


data = pandas.DataFrame(
    {
        'collateral_token_price': [
            x
            for x
            in decimal_range(
                # TODO: make it dependent on the collateral token .. use prices.prices[COLLATERAL_TOKEN]
                start = decimal.Decimal('1000'),
                stop = decimal.Decimal('3000'),
                # TODO: make it dependent on the collateral token
                step = decimal.Decimal('50'),
            )
        ]
    },
)
# TOOD: needed?
# data['collateral_token_price_multiplier'] = data['collateral_token_price_multiplier'].map(decimal.Decimal)
data['max_borrowings_to_be_liquidated'] = \
    data['collateral_token_price'].apply(
        lambda x: simulate_liquidations_under_absolute_price_change(
            prices = prices,
            collateral_token = COLLATERAL_TOKEN,
            collateral_token_price = x,
            state = state,
            borrowings_token = BORROWINGS_TOKEN,
        )
    )

data['max_borrowings_to_be_liquidated_at_interval'] = data['max_borrowings_to_be_liquidated'].diff().abs()
# TODO: drops also other NaN, if there are any
data.dropna(inplace = True)
data

In [ ]:
# Setup the AMM.
import swap_liquidity


jediswap = swap_liquidity.SwapAmm('JediSwap')
jediswap.add_pool('ETH', 'USDC', '0x04d0390b777b424e43839cd1e744799f3de6c176c7e32c1812a41dbd9c19db6a')
await jediswap.get_balance()


def get_amm_supply_at_price(
    collateral_token: str,
    collateral_token_price: decimal.Decimal,
    borrowings_token: str,
) -> decimal.Decimal:
    return jediswap.get_pool(collateral_token, borrowings_token).supply_at_price(borrowings_token, collateral_token_price)


data['amm_borrowings_token_supply'] = \
    data['collateral_token_price'].apply(
        lambda x: get_amm_supply_at_price(
            collateral_token = COLLATERAL_TOKEN,
            collateral_token_price = x,
            borrowings_token = BORROWINGS_TOKEN,
        )
    )
data

## Plot the liquidated and supply amounts

In [ ]:
import plotly.express


figure = plotly.express.bar(
    data.astype(float),
    x = 'collateral_token_price',
    y = ['max_borrowings_to_be_liquidated_at_interval', 'amm_borrowings_token_supply'],
    title = f'Potentially liquidatable amounts of {BORROWINGS_TOKEN} and the corresponding supply',
    barmode = 'overlay',
    opacity = 0.65,
)
figure.show()

# Debugging negative deposits below

TODO: remove later

In [ ]:
# TODO: negative deposits?
print("Action   Token   Raw value   Raw $   Readable $")
for attr in ['deposit', 'borrowings']:
    for token in ['ETH', 'wBTC', 'USDC', 'DAI', 'USDT']:
        values = [getattr(user_state.token_states[token], attr) for user, user_state in state.user_states.items()]
        print(attr, token, pandas.Series(values).min(), f"${prices.to_dollars(pandas.Series(values).min(), token)}", prices.to_dollars_pretty(pandas.Series(values).min(), token))

In [ ]:
# TODO: show negative deposits
[
    (user, user_state.token_states['USDT'].deposit)
    for user, user_state
    in state.user_states.items()
    if user_state.token_states['USDT'].deposit < 0
]